In [1]:
1+1

2

In [3]:
#https://alejandro-ao.com/chat-with-mysql-using-python-and-langchain/
#https://www.youtube.com/watch?v=9ccl1_Wu24Q
import sys
print(sys.version)


3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)]


In [5]:
print(sys.version_info)

sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
